In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
import copy
import seaborn as sns
import tensorflow as tf
from sklearn.linear_model import LinearRegression

## Dataset:

Dua, D. and Graff, C. (2019). UCI Machine Learning Repository [http://archive.ics.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

Source: Data Source: [http://data.seoul.go.kr/] SOUTH KOREA PUBLIC HOLIDAYS. URL: [publicholidays.go.kr]

In [ ]:
cols = ["bike_count", "hour", "temp", "humidity", "wind", "visibility", "dew_pt_temp", "radiation", "rain", "snow", "functional"]
url = "https://raw.githubusercontent.com/usernameneo/Data-Science-Projects/refs/heads/main/fcc/BIKESHARE/SeoulBikeData.csv"

In [ ]:
df = pd.read_csv(url).drop(["Date", "Holiday", "Seasons"], axis=1)
df.columns = cols
df["functional"] = (df["functional"] == "Yes").astype(int)
df.head()

The data shows the bike count across every hour. For simplicity, we'll index onto a single hour, 12 noon.

In [ ]:
df = df[df["hour"] == 12]
df = df.drop(["hour"], axis=1)
df.head()

In [ ]:
for label in df.columns[1:]:
    plt.scatter(df[label], df["bike_count"])
    plt.title(label)
    plt.ylabel("bike count at noon")
    plt.xlabel(label)
    plt.show()

In [24]:
df = df.drop(["wind", "visibility", "functional"], axis=1)
df.head()

,bike_count,temp,humidity,dew_pt_temp,radiation,rain,snow
12,449,1.7,23,-17.2,1.11,0.0,0.0
36,479,4.3,41,-7.8,1.09,0.0,0.0
60,333,5.8,85,3.4,0.43,0.0,0.0
84,393,-0.3,38,-12.9,1.11,0.0,0.0
108,321,-2.3,25,-19.7,0.00,0.0,0.0


## Train, validation and test data prep

In [28]:
df_train, df_val, df_test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])

/home/codespace/.python/current/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [29]:
def get_Xy(dataframe, y_label, x_labels=None):
    dataframe = copy.deepcopy(dataframe)
    if not x_labels:
        X = dataframe[[col for col in dataframe.colunms if col != y_label]].values
    else:
        if len(x_labels) == 1:
            X = dataframe[x_labels[0]].values.reshape(-1, 1)
        else:
            X = dataframe[x_labels].values

    y = dataframe[y_label].values.reshape(-1, 1)
    data = np.hstack((X, y))

    return data, X, y

In [32]:
_, X_train_temp, y_train_temp = get_Xy(df_train, "bike_count", x_labels=["temp"])
_, X_val_temp, y_val_temp = get_Xy(df_val, "bike_count", x_labels=["temp"])
_, X_test_temp, y_test_temp = get_Xy(df_test, "bike_count", x_labels=["temp"])